In [5]:
import pandas as pd
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
import datetime
from enum import Enum

In [6]:
df = pd.read_csv("../subscriptions.csv")
pay_df = pd.read_csv("../data/subscriptions_new.csv")

email = 'Customer Email'
name = 'Customer Name'
pay_df['cust_id'] = pay_df[name] + '-' + pay_df[email]
df['cust_id'] = df[name] + '-' + df[email]

pay_dict = pay_df[['cust_id', 'Amount']].to_dict(orient='tight', index=False)
pay_map = {i[0]: i[1] for i in pay_dict['data']}
# df['amount'] = df['cust_id'].map(pay_map)
# df = df.dropna(subset = "amount")

In [10]:
pay_df.columns

Index(['id', 'Customer ID', 'Customer Description', 'Customer Email', 'Plan',
       'Product', 'Product ID', 'Quantity', 'Currency', 'Cancellation Reason',
       'Interval', 'Amount', 'Status', 'Created (UTC)', 'Start (UTC)',
       'Start Date (UTC)', 'Current Period Start (UTC)',
       'Current Period End (UTC)', 'Trial Start (UTC)', 'Trial End (UTC)',
       'Application Fee Percent', 'Coupon', 'Tax Percent', 'Canceled At (UTC)',
       'Cancel At Period End', 'Ended At (UTC)', 'Exclusive Tax Rate',
       'Inclusive Tax Rate', 'Customer Name', 'Automatic Tax Enabled',
       'Tax Location Recognized', 'cust_id'],
      dtype='object')

In [19]:
import datetime
canceled_at_col = "Canceled At (UTC)"
ended_at_col = "Ended At (UTC)"
start_at_col = "Start Date (UTC)"

col = start_at_col
df[col] = pd.to_datetime(df[col], errors="coerce")
(df[col] > datetime.datetime.strptime("01/7/2025", "%d/%m/%Y")).sum()

6

In [8]:
pay_df[pay_df[name].str.contains('Ward')]

,id,Customer ID,Customer Description,Customer Email,Plan,Product,Product ID,Quantity,Currency,Cancellation Reason,...,Tax Percent,Canceled At (UTC),Cancel At Period End,Ended At (UTC),Exclusive Tax Rate,Inclusive Tax Rate,Customer Name,Automatic Tax Enabled,Tax Location Recognized,cust_id
95,sub_1QkBeNLLDYopDCcGJcklJl6f,cus_RdSZzUbP9eXGBb,NaN,alexjward79@gmail.com,price_1OH7q7LLDYopDCcGlwcnPAq2,Standard Membership: Group Lessons,prod_P5IQZ53aUYwioI,1,gbp,cancellation_requested,...,NaN,2025-01-31 19:11,False,2025-01-31 19:11,NaN,NaN,Mrs Alexandra J Ward,False,True,Mrs Alexandra J Ward-alexjward79@gmail.com
276,sub_1OH8fdLLDYopDCcGG78ZzjBf,cus_P5JIQv0vju3cGN,NaN,ningning77@hotmail.com,price_1OH7q7LLDYopDCcGlwcnPAq2,Standard Membership: Group Lessons,prod_P5IQZ53aUYwioI,1,gbp,NaN,...,NaN,2023-11-28 16:34,False,2023-11-28 16:34,NaN,NaN,Matthew Ward,False,True,Matthew Ward-ningning77@hotmail.com


In [9]:
df[~df['cust_id'].isin(pay_df.cust_id)]

,id,Customer ID,Customer Email,Quantity,Amount,Status,Created (UTC),Start (UTC),Start Date (UTC),Current Period Start (UTC),Current Period End (UTC),Trial Start (UTC),Trial End (UTC),Canceled At (UTC),Cancel At Period End,Ended At (UTC),Customer Name,cust_id


In [38]:
df['cust_id']

0                   Azlina Yeo-azlinayeo@googlemail.com
1                        Azlina Yeo-azlinayeo@gmail.com
2                      Yingshi Ji-yingshijinb@gmail.com
3                     Moshe Dabach-moshed88@hotmail.com
4                              y luk-lukyip27@gmail.com
                             ...                       
278                     Ankur Zadoo-ankurzadu@gmail.com
279      Bleine E D S Andrade-bleineandrade07@gmail.com
280            Sarah-Jane  Gamble -spiffymiss@gmail.com
281           Venkatesh Ramasamy-tutor4ruchit@gmail.com
282    Jayapriya Thanaraj-jayapriyathanaraj@hotmail.com
Name: cust_id, Length: 283, dtype: object

In [34]:
df['cust_id'].map(pay_map).value_counts(dropna=False)

cust_id
80.0      75
NaN       62
220.0     46
99.0      34
150.0     16
1080.0     8
139.0      7
149.0      5
129.0      4
180.0      4
1.0        4
225.0      3
110.0      3
60.0       3
420.0      2
540.0      2
200.0      2
1920.0     1
534.0      1
120.0      1
Name: count, dtype: int64

In [13]:
class PrivateLesson(Enum):
    Private_Month = 'Private-Month', 'Private', 1, 1, [129, 150, 160, 180, 220]
    # 110 - twice
    Private_Three_Months = 'Private_3_Months', 'Private', 3, 1, [504, 540]
    Private_Three_Months_Twice = 'Private_3_Months_Twice_week', 'Private', 3, 2, [1080]
    Private_Six_Months = 'Private_6_Months', 'Private', 6, 1, [1080, 840, 960]
    Private_Six_Months_Twice = 'Private_6_Months_Twice_week', 'Private', 6, 2, [1920, 2180]

    Group_Month = 'Group-Month', 'Group', 1, 1, [60, 80, 160, 240]
    Group_Month_Twice = 'Group-Month_Twice_week', 'Group',1, 2, [99]
    Group_Six_Months = 'Group_6_Months', 'Group', 6, 1, [420]
    Group_Six_Months_Twice = 'Group_6_Months_Twice_week', 'Group', 6, 2, [534]

    def __init__(self, label: str, class_type: str, months: int, times: int, cost_options: list):
        self.label = label
        self.class_type = class_type
        self.months = months
        self.times = times
        self.cost_options = cost_options

    def include(self, x:int):
        return x in self.cost_options


    @classmethod
    def from_label(cls, label):
        for item in cls:
            if item.label == label:
                return item


In [14]:
def find_class_type(x):
    lesson_types = [
    PrivateLesson.Private_Month,
    PrivateLesson.Private_Three_Months,
    PrivateLesson. Private_Three_Months_Twice,
    PrivateLesson.Private_Six_Months,
    PrivateLesson.Private_Six_Months_Twice,
    PrivateLesson.Group_Month,
    PrivateLesson.Group_Month_Twice,
    PrivateLesson.Group_Six_Months,
    PrivateLesson.Group_Six_Months_Twice
    ]
    for lesson in lesson_types:
        if lesson.include(x):
            return lesson
    print(f"missing amount {x}")
 

In [16]:
# missing amount 110.0  = twice 1 month
# missing amount 225.0  = 420 
# missing amount 139.0  = remove
# missing amount 120.0  = 129
# missing amount 149.0  = Group - 1 month

# missing amount 1.0 = remove




In [23]:
pay_df.query("Amount == 120")

,id,Customer ID,Customer Description,Customer Email,Plan,Product,Product ID,Quantity,Currency,Cancellation Reason,...,Tax Percent,Canceled At (UTC),Cancel At Period End,Ended At (UTC),Exclusive Tax Rate,Inclusive Tax Rate,Customer Name,Automatic Tax Enabled,Tax Location Recognized,cust_id
275,sub_1OI9CdLLDYopDCcG7rCwHgM4,cus_P6LuEXj5ebpQhK,NaN,nadjete2003@gmail.com,price_1OHtCHLLDYopDCcGCpZuudVa,"One-to-One membership, reduced hourly rate",prod_P65NUIzI9gSoLk,1,gbp,cancellation_requested,...,NaN,2024-10-07 13:45,False,2024-10-07 13:45,NaN,NaN,N HOUAIDJI,False,True,N HOUAIDJI-nadjete2003@gmail.com


In [18]:
df['class'] = df['amount'].apply(find_class_type)

missing amount 110.0
missing amount 225.0
missing amount 225.0
missing amount 1.0
missing amount 139.0
missing amount 149.0
missing amount 110.0
missing amount 149.0
missing amount 149.0
missing amount 139.0
missing amount 110.0
missing amount 139.0
missing amount 139.0
missing amount 139.0
missing amount 139.0
missing amount 225.0
missing amount 1.0
missing amount 1.0
missing amount 149.0
missing amount 139.0
missing amount 1.0
missing amount 200.0
missing amount 149.0
missing amount 120.0
missing amount 200.0


In [22]:
df['class'].iloc[0].label

'Private-Month'

In [27]:
pay_df.columns

Index(['id', 'Customer ID', 'Customer Description', 'Customer Email', 'Plan',
       'Product', 'Product ID', 'Quantity', 'Currency', 'Cancellation Reason',
       'Interval', 'Amount', 'Status', 'Created (UTC)', 'Start (UTC)',
       'Start Date (UTC)', 'Current Period Start (UTC)',
       'Current Period End (UTC)', 'Trial Start (UTC)', 'Trial End (UTC)',
       'Application Fee Percent', 'Coupon', 'Tax Percent', 'Canceled At (UTC)',
       'Cancel At Period End', 'Ended At (UTC)', 'Exclusive Tax Rate',
       'Inclusive Tax Rate', 'Customer Name', 'Automatic Tax Enabled',
       'Tax Location Recognized', 'cust_id'],
      dtype='object')

In [28]:
a = [
#0.17307692307692307,
0.08196721311475409,
0.09836065573770492,
0.06944444444444445,
0.09090909090909091,
0.058823529411764705,
#0.13793103448275862,
0.036585365853658534,
0.07407407407407407,
0.05128205128205128,
    
]


In [29]:
sum(a) / len(a)

0.07018080310344287